In [2]:
import pandas as pd
data = pd.read_csv("/home/caro/Documents/curso_geopython/Final-exercise/data/2315676.txt",delim_whitespace=True, 
                   na_values=[-9999], skiprows = [1],low_memory=False)

In [106]:
data

,STATION,STATION_NAME,DATE,TAVG,TMAX,TMIN,TAVG2
0,GHCND:FI000007501,SODANKYLA-AWS-FI,19080101,NaN,2.0,-37.0,-17.5
1,GHCND:FI000007501,SODANKYLA-AWS-FI,19080102,NaN,6.0,-26.0,-10.0
2,GHCND:FI000007501,SODANKYLA-AWS-FI,19080103,NaN,7.0,-27.0,-10.0
3,GHCND:FI000007501,SODANKYLA-AWS-FI,19080104,NaN,-3.0,-27.0,-15.0
4,GHCND:FI000007501,SODANKYLA-AWS-FI,19080105,NaN,4.0,-36.0,-16.0
...,...,...,...,...,...,...,...
41060,GHCND:FI000007501,SODANKYLA-AWS-FI,20201003,47.0,51.0,NaN,47.0
41061,GHCND:FI000007501,SODANKYLA-AWS-FI,20201004,43.0,47.0,37.0,43.0
41062,GHCND:FI000007501,SODANKYLA-AWS-FI,20201005,42.0,NaN,37.0,42.0
41063,GHCND:FI000007501,SODANKYLA-AWS-FI,20201006,45.0,46.0,43.0,45.0


In [ ]:
## item 2

In [105]:
nulls = data["TAVG"].isna()
data["TAVG2"]=(data["TMAX"]+data["TMIN"])/2
for i, row in data.iterrows():
    if nulls[i]==False:
        #print(i)
        #print(nulls[i])
        data.at[i,"TAVG2"]=data.at[i,"TAVG"]

In [ ]:
## item 3

In [116]:
def fahr_to_celsius(temp_fahrenheit):
    return (temp_fahrenheit - 32)/1.8

In [120]:
data["TAVG2_C"]=round(data["TAVG2"].apply(fahr_to_celsius),1)

In [123]:
data["TMAX_C"]=round(data["TMAX"].apply(fahr_to_celsius),1)

In [122]:
data["TMIN_C"]=round(data["TMIN"].apply(fahr_to_celsius),1)

In [ ]:
## item 4

In [128]:
data["YEAR"] = data["DATE"].astype(str).str.slice(start=0,stop=4)

In [130]:
data["MO"] = data["DATE"].astype(str).str.slice(start=4,stop=6)

In [133]:
for i, row in data.iterrows():
    if data.at[i,"MO"]=='12' or data.at[i,"MO"]=='01' or data.at[i,"MO"]== '02':
        data.at[i,"SEAS"]='WI'
    elif data.at[i,"MO"]=='03' or data.at[i,"MO"]=='04' or data.at[i,"MO"]== '05':
        data.at[i,"SEAS"]='SP'
    elif data.at[i,"MO"]=='06' or data.at[i,"MO"]=='07' or data.at[i,"MO"]== '08':
        data.at[i,"SEAS"]='SU'
    elif data.at[i,"MO"]=='09' or data.at[i,"MO"]=='10' or data.at[i,"MO"]== '11':
        data.at[i,"SEAS"]='AU'